In [21]:
from scipy.spatial.distance import cosine
import numpy as np
import pickle
from tqdm.notebook import tqdm

In [3]:
with open('train_data/training_instruction_sen_emb_phrase_chunk_hard_example_full.pkl', 'rb') as f:
    data = pickle.load(f)

In [19]:
def calculate_average_hard_negative_cosine_similarity(val, want_print = False):
    original_emb = val['embedding']
    verb_polluted_emb_arr = val['action_polluted_embedding']
    noun_polluted_emb_arr = val['noun_polluted_embedding']
    # calculate action_polluted_emb
    verb_cosine_score = 0
    if verb_polluted_emb_arr is None:
        verb_cosine_score = 1
    else:
        for verb_polluted_emb in verb_polluted_emb_arr:
            verb_cosine_score += (1 - cosine(original_emb, verb_polluted_emb))
        verb_cosine_score /= len(verb_polluted_emb_arr)
    
    noun_cosine_score = 0
    if noun_polluted_emb_arr is None:
        noun_cosine_score = 1
    else:
        for noun_polluted_emb in noun_polluted_emb_arr:
            noun_cosine_score += (1 - cosine(original_emb, noun_polluted_emb))
        noun_cosine_score /= len(noun_polluted_emb_arr)
        
    if want_print:
        print(f'verb cosine similarity: {verb_cosine_score}')
        print(f'noun cosine similarity: {noun_cosine_score}')
    return verb_cosine_score, noun_cosine_score
    
    
    

In [30]:
def calculate_cosine_similarity_of_in_batch_negative(key, val, data, want_print=False):
    cosine_score = 0 
    original_emb = val['embedding']
    for k, v in tqdm(data.items(), leave=False):
        if key == k:
            continue 
        in_batch_neg_emb = v['embedding']
        cosine_score += (1 - cosine(original_emb, in_batch_neg_emb))
    cosine_score /= (len(data) - 1)
    if want_print:
        print(f"normal negative cosine similarity: {cosine_score}")
    return cosine_score

In [15]:
calculate_average_hard_negative_cosine_similarity(data[2], True)

verb cosine similarity: 0.865441610455513
noun cosine similarity: 0.646878668665886


(0.865441610455513, 0.646878668665886)

In [22]:
verb_hard_negative_cosine_similarity = 0
noun_hard_negative_cosine_similarity = 0


for key, val in tqdm(data.items()):
    verb_cosine_score, noun_cosine_score = calculate_average_hard_negative_cosine_similarity(val)
    verb_hard_negative_cosine_similarity += verb_cosine_score
    noun_hard_negative_cosine_similarity += noun_cosine_score
    
verb_hard_negative_cosine_similarity /= len(data)
noun_hard_negative_cosine_similarity /= len(data)

print(f'Trainig data average verb polluted negatives cosine similarity: {verb_hard_negative_cosine_similarity}')
print(f'Trainig data average noun polluted negatives cosine similarity: {noun_hard_negative_cosine_similarity}')

  0%|          | 0/5655 [00:00<?, ?it/s]

/home/sukai/anaconda3/envs/atari_clean/lib/python3.8/site-packages/scipy/spatial/distance.py:620: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


Trainig data average verb polluted negatives cosine similarity: 0.8543671234110198
Trainig data average noun polluted negatives cosine similarity: 0.8098746713161679


In [25]:
calculate_cosine_similarity_of_in_batch_negative(3, data[3], data, True)

  0%|          | 0/5655 [00:00<?, ?it/s]

normal negative cosine similarity: 0.5812674068259738


0.5812674068259738

In [26]:
lowest_verb_hard_negative_cosine_similarity = 100
lowest_noun_hard_negative_cosine_similarity = 100


for key, val in tqdm(data.items()):
    verb_cosine_score, noun_cosine_score = calculate_average_hard_negative_cosine_similarity(val)
    if verb_cosine_score < lowest_verb_hard_negative_cosine_similarity:
        lowest_verb_hard_negative_cosine_similarity = verb_cosine_score
        lowest_verb_hard_negative_cosine_similarity_key = key
    if noun_cosine_score < lowest_noun_hard_negative_cosine_similarity:
        lowest_noun_hard_negative_cosine_similarity = noun_cosine_score
        lowest_noun_hard_negative_cosine_similarity_key = key


print(f'lowest verb polluted negatives cosine similarity: {lowest_verb_hard_negative_cosine_similarity},({lowest_verb_hard_negative_cosine_similarity_key})')
print(f'lowest noun polluted negatives cosine similarity: {lowest_noun_hard_negative_cosine_similarity}, ({lowest_noun_hard_negative_cosine_similarity_key})')

  0%|          | 0/5655 [00:00<?, ?it/s]

/home/sukai/anaconda3/envs/atari_clean/lib/python3.8/site-packages/scipy/spatial/distance.py:620: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


lowest verb polluted negatives cosine similarity: 0.49136530929803846,(969)
lowest noun polluted negatives cosine similarity: 0.4409869178533554, (1531)


In [27]:
calculate_cosine_similarity_of_in_batch_negative(969, data[969], data, True)

  0%|          | 0/5655 [00:00<?, ?it/s]

normal negative cosine similarity: 0.513528911846874


0.513528911846874

In [32]:
data[969]['spell_correction_txt']

're enter the'

In [28]:
calculate_cosine_similarity_of_in_batch_negative(1531, data[1531], data, True)

  0%|          | 0/5655 [00:00<?, ?it/s]

normal negative cosine similarity: 0.43746678093675595


0.43746678093675595

In [31]:
# calculate how many examples that the difference between normal negative cosine score and hard negative cosine score
# are within 0.1
count = 0 
for key, val in tqdm(data.items()):
    verb_cosine_score, noun_cosine_score = calculate_average_hard_negative_cosine_similarity(val)
    in_batch_neg_cosine_score = calculate_cosine_similarity_of_in_batch_negative(key, val, data)
    if np.abs(in_batch_neg_cosine_score - verb_cosine_score) < 0.1 \
    or np.abs(in_batch_neg_cosine_score - noun_cosine_score) < 0.1:
        count += 1
print(f"there are {count} examples that has similar hard negative and in-batch negative cosine score")

  0%|          | 0/5655 [00:00<?, ?it/s]

  0%|          | 0/5655 [00:00<?, ?it/s]

  0%|          | 0/5655 [00:00<?, ?it/s]

/home/sukai/anaconda3/envs/atari_clean/lib/python3.8/site-packages/scipy/spatial/distance.py:620: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


  0%|          | 0/5655 [00:00<?, ?it/s]

...

  0%|          | 0/5655 [00:00<?, ?it/s]

  0%|          | 0/5655 [00:00<?, ?it/s]

  0%|          | 0/5655 [00:00<?, ?it/s]

  0%|          | 0/5655 [00:00<?, ?it/s]

there are 1416 examples that has similar hard negative and in-batch negative cosine score
